# 1. Introduction

In [ ]:
import numpy as np
import pandas as pd
import warnings
import time

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px # treemap
from wordcloud import WordCloud, STOPWORDS # cloud map
from mpl_toolkits.mplot3d import Axes3D # 3D plot

# market basket analysis
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans # cluster


------------

---------------

# 2. Data Cleaning

In [ ]:
# ignore warning
warnings.filterwarnings("ignore")

df=pd.read_csv("/kaggle/input/whole-foods-transaction-data/Whole_Foods_Transaction_Data.csv", encoding='latin1')
df

In [ ]:
df.info()

- department_id may contains strings

In [ ]:
distinct_values = df['department_id'].unique()
distinct_values

- 'household' and 'personal care' seem to be recorded differently
- Some numbers are recorded as str

In [ ]:
filtered_rows = df[(df['department_id'] == 'household') | (df['department_id'] == 'personal care')]
filtered_rows

- Only 5 rows have these labels
- There are some weird strings in 'product_name'
- And they are all null in 'aisle'

In [ ]:
df.isna().sum()

- Now we know those only 5 rows of null are here
- Just seem like bad data considering no aisle numbers, weird names, and different recording rule
- Let's just remove them

In [ ]:
# drop na, and convert the strings in 'department_id' to int
df = df.dropna()
df['department_id'] = df['department_id'].astype(int)

In [ ]:
df.info()

In [ ]:
# just checking 'department_id'
distinct_values1 = sorted(df['department_id'].unique())
distinct_values1

In [ ]:
# checking how many products we are dealing with
distinct_product_name = df['product_name'].unique()
distinct_department = df['department'].unique()
distinct_aisle = df['aisle'].unique()
print(f"Number of unique product names: {len(distinct_product_name)}")
print(f"Number of unique department: {len(distinct_department)}")
print(f"Number of unique aisle: {len(distinct_aisle)}")

In [ ]:
# check for duplicate rows
duplicates = len(df[df.duplicated()])
duplicates

In [ ]:
df.to_csv('WholeFoods_Market_Basket.csv', index=False)

### Short Summary
- This dataset contains 250000 rows and 8 columns
- 5 null value rows were removed because of their suspicious names and unusual record formats
- No duplicated data
- We have 23321 distinct products
- 21 departments (AKA categories)
- And 134 aisles

-----------------------------

------------------------------------------------------------------------------------

# 3. EDA & Visuals

In [ ]:
# pie chart
sns.set_palette("pastel")
plt.pie(df['reordered'].value_counts(),labels=df['reordered'].value_counts().index,autopct='%1.1f%%')
plt.title('Total Distribution of Reordered')
plt.show()

In [ ]:
# stacked bar
percentage_df = df.groupby('department')['reordered'].value_counts(normalize=True).unstack().fillna(0) * 100
percentage_df = percentage_df[[1, 0]]
# Sort the data by the highest percentage of '1s'
sorted_df = percentage_df.sort_values(by=1, ascending=False)

# Plot stacked bar chart
ax = sorted_df.plot(kind='bar', stacked=True, figsize=(10, 6))

# Set labels and title
ax.set_xlabel('Department')
ax.set_ylabel('Percentage')
ax.set_title('Average Percentage of Reordered by Category')

plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels if needed
plt.legend(title='Reordered')
plt.tight_layout()
plt.show()

In [ ]:
# data for scatter plot
grouped = df.groupby(['product_name','department']).agg(proportion_reordered=('reordered', 'mean'),
                                                        count=('product_name', 'size'))
grouped = grouped.reset_index()

# only show products higher than 100 counts, then sort
product_frequency = grouped[grouped['count'] > 100].sort_values(by='proportion_reordered', ascending=False)

In [ ]:
# data for table plot
top_20_products = product_frequency.head(20)

# keep only 3 decimals
top_20_products['proportion_reordered'] = top_20_products['proportion_reordered'].apply(lambda x: f'{x:.3f}')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11, 12))

# first plot
sns.scatterplot(x=product_frequency['proportion_reordered'], y=product_frequency['count'], hue=product_frequency['department'], palette='Set2', ax=axes[0],s=70)
axes[0].set_title('Product Frequency vs. Proportion Reordered (Higher Than 100 Counts)')
axes[0].set_xlabel('Proportion Reordered')
axes[0].set_ylabel('Product Frequency')

# second plot
axes[1].axis('off')  # turn off the grid

table = axes[1].table(cellText=top_20_products.values, colLabels=top_20_products.columns,loc='center', cellLoc='left')#
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.4, 1.2)  

plt.subplots_adjust(hspace=0.3)
plt.tight_layout()
plt.show()

- Most products (purchased at least 100 times or more) are clustered around a reorder rate between 0.5 and 0.7
- Most products are purchased less than 500 times
- The purchase quantity of produce shows a clear fan-shaped dispersion pattern in relation to the reorder rate
- The second discernible fan-shaped dispersion is dairy eggs 
- Meaning some of the produce and dairy eggs not only have a higher purchase frequency,but also an increased reorder rate, while other products do not exhibit this pattern

In [ ]:
niche_product_frequency = grouped[(grouped['count'] >= 5) & 
                                  (grouped['count'] <= 100) & 
                                  (grouped['proportion_reordered'] > 0.8)].sort_values(by=['proportion_reordered', 'count'], 
                                                                                       ascending=[False, True])
niche_product_frequency = niche_product_frequency.reset_index()


plt.figure(figsize=(7, 6))
sns.scatterplot(x=niche_product_frequency['proportion_reordered'], y=niche_product_frequency['count'], 
                hue=niche_product_frequency['department'], palette='Set2',s=70)

plt.title('Niche Product Frequency vs. Proportion Reordered (Between 5-100 Counts)')
plt.xlabel('Proportion Reordered')
plt.ylabel('Product Frequency')

plt.tight_layout()
plt.show()

- No clear patterns observed for products that are sold less than 100 counts and have a reorder rate higher than 80%

In [ ]:
department_counts = df['department'].value_counts().sort_values(ascending=False)

department_counts_df = department_counts.reset_index()
department_counts_df.columns = ['department', 'count']

warnings.filterwarnings('ignore', category=DeprecationWarning)
sns.barplot(data=department_counts_df, x='department', y='count',palette='plasma_r') # reverse the palette

plt.xticks(rotation=-45, ha='left')  
plt.xlabel('Department')  
plt.ylabel('Count')  
plt.title('Frequency of Categories') 
plt.grid(axis='y', linestyle='--', alpha=0.4) 
plt.gca().set_axisbelow(True) # set grid to the background
plt.show()


In [ ]:
aisle_counts = df['aisle'].value_counts().sort_values(ascending=False)

aisle_counts_df = aisle_counts.reset_index()
aisle_counts_df.columns = ['aisle', 'count']


sns.barplot(data=aisle_counts_df.head(20), x='aisle', y='count',palette='plasma_r') # reverse the palette

plt.xticks(rotation=-45, ha='left')  
plt.xlabel('aisle')  
plt.ylabel('Count')  
plt.title('Frequency of Top 20 Aisle') 
plt.grid(axis='y', linestyle='--', alpha=0.4) 
plt.gca().set_axisbelow(True) # set grid to the background
plt.show()

In [ ]:
df1=df.copy()
# group by each 'product_name', count occurrences, create a corresponding column for the count
df1['count'] = df1.groupby('product_name')['product_name'].transform('count')

# sort based on the values in count col
sorted_df1 = df1.sort_values(by='count', ascending=False)

# select only the 3 cols we need
sdf = sorted_df1[['product_name','department','count']]

# keep only the unique rows we need
product_counts_df = sdf.drop_duplicates(subset=['product_name'], keep='first')


# set a canvas size ax
fig, ax = plt.subplots(figsize=(12, 6))
# dodge=False,keeps the bars fixed in their position
sns.barplot(data=product_counts_df.head(40), x='product_name', y='count', hue='department', dodge=False, ax=ax) 

plt.xticks(rotation=-45, ha='left')  
plt.xlabel('Product Name')  
plt.ylabel('Count')  
plt.title('Frequency of Top 40 Products') 
plt.grid(axis='y', linestyle='--', alpha=0.4) 
plt.gca().set_axisbelow(True) # set grid to the background
plt.show()

In [ ]:
# prepare the data for a treemap
# group the df by 'department' and 'aisle', then calculate the size of each group and reset the index with a column named 'count'
grouped_data1 = df.groupby(['department', 'aisle']).size().reset_index(name='count')

# calculate the total count of each department by summing up the 'count' column after grouping by 'department'
department_counts_sorted1 = grouped_data1.groupby('department')['count'].sum().reset_index().sort_values(by='count', ascending=False)

# merge the sorted department counts back with the grouped data based on the 'department' column
sorted_grouped_data = pd.merge(department_counts_sorted1, grouped_data1, on='department')

# select only the relevant columns 'department', 'aisle', and 'count_y' (which is the count from the grouped data)
sorted_grouped_data1 = sorted_grouped_data[['department', 'aisle', 'count_y']]

# rename the columns to match the desired format
sorted_grouped_data1.columns = ['department', 'aisle', 'count']
sorted_grouped_data1

In [ ]:
palette = px.colors.qualitative.Set3

fig = px.treemap(sorted_grouped_data1, 
                 path=['department', 'aisle'],  
                 values='count',  
                 color='department',
                 color_discrete_sequence=palette,
                 #color='count', 
                 title='Treemap of Product Frequency by Department and Aisle') #color_continuous_scale= 'viridis',
fig.update_layout(width=1000,
                  height=800,
                  margin = dict(t=50, l=50, r=50, b=50),
                  uniformtext=dict(minsize=12))
                  #uniformtext=dict(minsize=10)
fig.update_traces(hoverinfo='skip', hovertemplate=None, # turn off hover info to save memory
                  marker=dict(cornerradius=5, # rounded corners
                               line=dict(width=0.5, color='white')))
fig.show() 

In [ ]:
palette = px.colors.qualitative.Set3 # color:optional

# Create sunburst chart using Plotly Express
fig = px.sunburst(sorted_grouped_data1, 
                  path=['department', 'aisle'],  
                  values='count',  
                  color='department', # color:optional
                  color_discrete_sequence=palette, # color:optional
                  title='Sunburst of Product Frequency by Department and Aisle')

# Update layout
fig.update_traces(hoverinfo='skip', hovertemplate=None) # turn off hover info to save memory
fig.update_layout(width=1000, height=800)

fig.show()

### Short Summary
- Banana ...


-----------

-------------

# 3. Modeling

### Data Preparation

In [ ]:
df

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# select only 2 relevant cols 'transaction_id', 'product_name', then copy as our new df2
df2 = df[['transaction_id', 'product_name']].copy()
# aggregate the products together, separate by ^ (beacuse there are !,$,%,and commas in the product names) 
df_grouped = df2.groupby('transaction_id')['product_name'].agg('^'.join).reset_index()
df_grouped.head()

In [ ]:
# then convert to lists of products, separated by commma
product_list = df_grouped["product_name"].str.split('^').tolist()

In [ ]:
# one hot encoding
temp = TransactionEncoder()

# use sparse format to save memory (makes the model run faster)
onehot_df = temp.fit(product_list).transform(product_list, sparse=True) 

# convert to DF using sparse format
encoded_df = pd.DataFrame.sparse.from_spmatrix(onehot_df,columns=temp.columns_)
encoded_df.head()

-------

### Association Rule with FP-Growth (Frequent Pattern Growth) Algorithm

Alternative algorithms are: 
- Apriori algorithm:  
> (An inferior version of FP-Growth) A classic pattern mining algorithm, employs the "apriori principle," which states that if an itemset is frequent, then all of its subsets must also be frequent. But this can suffer from scalability issues, especially with large datasets
- Eclat algorithm:  
> The Eclat algorithm is another frequent pattern mining algorithm. It uses a vertical data format to represent transaction databases, enabling efficient data processing and discovery of frequent itemsets in memory.
- PrefixSpan algorithm:  
> PrefixSpan is a recursive depth-first sequential pattern mining algorithm. It is used for mining frequent sequential patterns in sequence-type data and has advantages in handling such data.
- GSP algorithm (Generalized Sequential Pattern):  
> The GSP algorithm is used for mining sequential patterns similar to PrefixSpan. It is effective in handling sequence-type data and discovering frequent sequential patterns.

In [ ]:

start_time = time.time()

# find frequent itmes using FP GROWTH algorithm with minimum support of 0.0003 
frequent_itemsets = fpgrowth(encoded_df, min_support=0.0003, use_colnames=True) 

# add a new column 'length' to represent the number of items in each set
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# sort by support 
frequent_itemsets = frequent_itemsets.sort_values(by="support", ascending=False)
frequent_itemsets.head()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime: {elapsed_time:.1f} seconds")

In [ ]:
# check itemsets with 2 or more items sorted by support
frequent_itemsets1 = frequent_itemsets[frequent_itemsets['length'] >=2]
frequent_itemsets1.head()

In [ ]:
sns.barplot(data=frequent_itemsets1.head(20), x='support', y='itemsets',color='skyblue')
plt.title("Top 20 Frequent Itemsets")
plt.show()

- Highest support rate itemsets are almost all produce

In [ ]:
# since we need to find the associated itemsets, let's choose rows with lift ratio higher than 1.2
rules = association_rules(frequent_itemsets, metric= "lift", min_threshold = 1.2) 
# sort
rules = rules.sort_values(['lift','confidence','support'], ascending = False) 
rules

- Some explanations for the useful metrics here (using 3nd row as example):
1. **Support**: 
> - The probability of 'Baby Food Pears Squash' appearing together with 'Bananas' and 'Raspberries & Oats Organic Baby Food' is 0.03% among all transcations
2. **Confidence**: 
> - When customers purchase 'Baby Food Pears Squash', 73% of them also buy 'Bananas' and 'Raspberries & Oats Organic Baby Food' simultaneously
3. **Lift**: 
> - This measures the improvement in the occurrence probability of the consequent relative to random chance in a rule      
> - This means that the occurrence of X and Y together exceeds the occurrence of Y alone, indicating that X has a promoting effect on Y    
> - Higher than 1 indicates that purchasing 'Baby Food Pears Squash' positively influences the likelihood of purchasing 'Bananas' and 'Raspberries & Oats Organic Baby Food', showing a stronger association than expected by chance  
> - Close to 1 means no association
> - Between 0 and 1 means negative association
4. **Conviction**:
> - This assesses the independence of the consequent in the rule  
> - High conviction value indicates that the rule X => Y is highly dependent, and X rarely occurs without Y   
> - Higher than 1 indicates the presence of 'Bananas' and 'Raspberries & Oats Organic Baby Food' together is dependent on the presence of 'Baby Food Pears Squash', implying a strong association   
> - Equals to 1 means no association, the presence of X is purely independent of Y  
> - (Apple devices => Apple chargers are a high conviction pair because they can only be used together. If one day their relationship becomes low conviction, maybe Apple chargers will also be able to charge Android devices! XD)
5. **Zhangs_metric**:
> - This combines the strengths of Lift and Conviction  
> - Closer to 1 means strong positive association  
> - 0 means no association  
> - Closer to -1 means strong negative association  
> - In our case, 'Bananas' and 'Raspberries & Oats Organic Baby Food' together is strongly purchased with 'Raspberries & Oats Organic Baby Food' 
- When a rule is low confidence but high zhangs metric: This means that although this combination is not common, it may indicate a purchasing pattern of a specific group of customers


In [ ]:
rules.describe()

- Recall that we chose rules with a support rate higher than 0.2% and a lift ratio higher than 1, which returned 1300~ rules
- Since zhangs metric combines the strengths of Lift and Conviction, we will simply stick with zhangs metric for the evaluatiion of association


In [ ]:
df.to_csv('rules.csv', index=False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1)
plt.hist(rules['support'], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Support')
plt.ylabel('Frequency')
plt.title('Distribution of Support')

# confidence 
plt.subplot(2, 2, 2)
plt.hist(rules['confidence'], bins=20, color='salmon', edgecolor='black')
plt.xlabel('Confidence')
plt.ylabel('Frequency')
plt.title('Distribution of Confidence')

# zhangs_metric 
plt.subplot(2, 2, 3)
plt.hist(rules['zhangs_metric'], bins=20, color='lightgreen', edgecolor='black')
plt.xlabel("Zhang's Metric")
plt.ylabel('Frequency')
plt.title("Distribution of Zhang's Metric")

plt.tight_layout()
plt.show()

- Or we can plot some violin charts as below

In [ ]:
plt.figure(figsize=(10, 6))

# Support
plt.subplot(2, 2, 1)
sns.violinplot(x=rules['support'], color='skyblue')
plt.xlabel('Support')
plt.ylabel('Frequency')
plt.title('Distribution of Support')

# Confidence
plt.subplot(2, 2, 2)
sns.violinplot(x=rules['confidence'], color='salmon')
plt.xlabel('Confidence')
plt.ylabel('Frequency')
plt.title('Distribution of Confidence')

# Zhang's Metric
plt.subplot(2, 2, 3)
sns.violinplot(x=rules["zhangs_metric"], color='lightgreen')
plt.xlabel("Zhang's Metric")
plt.ylabel('Frequency')
plt.title("Distribution of Zhang's Metric")

plt.tight_layout()
plt.show()

- Most rules are clustered as low confidence when zhangs metric are somewhat clustered between 0.5 to 1.0
> Because we are trying so many combinations, and most of them are no better than random combinations, therefore low in confidence and support
- When an event occurs infrequently, its confidence may be low even if there is a strong association with other events
- This indicates some rules are rare sales or low sales volume

In [ ]:
sns.scatterplot(data=rules, x='support', y='confidence',hue='zhangs_metric', palette='viridis')

plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Rules with 3 Metrics')

plt.tight_layout()
plt.show()

- Right corner: Most of the high-frequency products (high support) belong to the confidence range of 0.1 to 0.3 with an average Zhang's Metric  
  > This indicats that those products that are often bought, although they are frequently seen, are not highly related to other products
- Top left corner: Most of the products with high association (high Zhang's Metric) belong to the low support range with a high acceptable level of confidence  
  > This indicats that those products although not common, are likely to be bought when they appear
- Bottom left corner: Most of rules are clustered around this area. These rules are not helpful, beacuse they are low in all 3 metrics
- Seems like the dots can be clustered into different groups, let's try clustering them
    

---

### Clustering with K-Means

- Can use hierarchical cluster or elbow plot to get the number of clusters for kmeans
- But we need to scale the data, since the max value for confidence is 0.5, but the max value for support is only 0.018

In [ ]:
# scale the data
rules_scale=rules[['confidence','support','zhangs_metric']].copy()
scale_cloumns = ['confidence','support','zhangs_metric']

scaler = StandardScaler()
 
rules_scale[scale_cloumns] = scaler.fit_transform(rules_scale[scale_cloumns])
#rules_scale

In [ ]:
# check elbow plot for the number if k
inertia_values = []
for k in range(1, 16):  # try different K values from 1 to 15
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(rules_scale)
    inertia_values.append(kmeans.inertia_)

In [ ]:
#plt.figure(figsize=(8, 6))
plt.plot(range(1, 16), inertia_values, marker='o')#
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Plot for K-means Clustering')
plt.xticks(range(1, 16))
plt.grid(True)
plt.show()

- K should either be 4 or 8, since clustering all the dots into 4 groups is not going to help with interpretation, we pick k=8

In [ ]:
# kmeans cluster fit
kmeans = KMeans(n_clusters=8)
kmeans.fit(rules_scale)

# cluster numbers
y_kmeans = kmeans.predict(rules_scale)

# add cluster numbers into the dataset
rules_scale['cluster'] = y_kmeans

In [ ]:
scatter1= plt.scatter(data=rules_scale ,x='support', y='confidence', c='cluster', s=50, cmap='viridis')#
center1 = kmeans.cluster_centers_
plt.scatter(center1[:, 1], center1[:, 0], c='red', edgecolors='black',s=200, alpha=0.75)

plt.xlabel('Support Level')
plt.ylabel('Confidence Level')
plt.title("Clustering into 8 Groups: Angle one")

legend1 = plt.legend(*scatter1.legend_elements(), title='Cluster')
plt.show()

In [ ]:
# scatter plot
scatter2 = plt.scatter(data=rules_scale, x='confidence', y="zhangs_metric", c='cluster', s=50, cmap='viridis')

# cluster centers
plt.scatter(center1[:, 0], center1[:, 2], c='red', edgecolors='black', s=200, alpha=0.75)

plt.xlabel('Confidence Level')
plt.ylabel("Zhang's Level")
plt.title('Clustering into 8 Groups: Angle Two')

# legend
legend2 = plt.legend(*scatter2.legend_elements(), title='Cluster')
plt.show()


- Big red dots are the centers of cluster
- 2D scatter plot can not show 3 metrics together. Let's check on 3D scatter plot

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# scatter plot for rules_scale
scatter3= ax.scatter(rules_scale['support'], rules_scale['confidence'], rules_scale['zhangs_metric'], cmap='Set1',c=rules_scale['cluster'], s=50)

# cluster centers
ax.scatter(center1[:, 1], center1[:, 0], center1[:, 2], c='red', edgecolors='black',s=200, alpha=0.75, label='Cluster Centers')

ax.set_xlabel('Support Level')
ax.set_ylabel('Confidence Level')
ax.set_zlabel("Zhang's Metric Level")
ax.set_title('Top Left Angle 3D Scatter Plot')

# legend
legend3 = ax.legend(*scatter3.legend_elements(), title='Cluster')
ax.add_artist(legend3)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8))

ax1 = fig.add_subplot(111, projection='3d')
scatter3_1 = ax1.scatter(rules_scale['support'], rules_scale['confidence'], rules_scale['zhangs_metric'], cmap='Set1', c=rules_scale['cluster'], s=50)
ax1.scatter(center1[:, 1], center1[:, 0], center1[:, 2], c='red', edgecolors='black', s=200, alpha=0.75, label='Cluster Centers')
ax1.set_xlabel('Support Level')
ax1.set_ylabel('Confidence Level')
ax1.set_zlabel("Zhang's Metric Level")
ax1.set_title('Top Right Angle 3D Scatter Plot')
legend3_1 = ax1.legend(*scatter3_1.legend_elements(), title='Cluster')
ax1.add_artist(legend3_1)
# tilt the angle
ax1.view_init(elev=15, azim=45)

plt.tight_layout()
plt.show()


---

### Pruning: 1st Round

- After checking all clusters, I realized that there are always bananas, avocados, and other similar produce items in each cluster, serving as nothing but noise. 
- Therefore, I choose to redo all the steps excluding the frequent products belong to the top 3 aisles to avoid strongly frequent products such as bananas
- Steps below are just repetitions from above

In [ ]:
# convert top 3 aisle names to a list
top_aisle_list = aisle_counts_df['aisle'].head(3).tolist()

# top_aisle_list
top_product_list = df[df['aisle'].isin(top_aisle_list)]['product_name'].tolist()

#top_product_list

- Create a function to implement the steps in the previous cell (for pruning)

In [ ]:
def remove_products_rules(rules, top_product_list, zhangs_threshold, df_name):
    def contains_top_product(fset):
        return any(item in fset for item in top_product_list)

    # use apply() to pass the values in 'antecedents' and 'consequents' to the function
    antecedents_contains = rules['antecedents'].apply(contains_top_product)
    consequents_contains = rules['consequents'].apply(contains_top_product)

    # remove the rows with those values
    notop_rules_all_zhangs = rules[~(antecedents_contains | consequents_contains)]

    # choose only rules with zhangs metric higher than the threshold
    notop_rules = notop_rules_all_zhangs[notop_rules_all_zhangs['zhangs_metric'] > zhangs_threshold]

    # assign the filtered DataFrame to the specified name
    globals()[df_name] = notop_rules

    return notop_rules

# eg: remove_products_rules(rules, top_product_list, 0.5, 'notop_rules')

In [ ]:
remove_products_rules(rules, top_product_list, 0.5, 'notop_rules')

In [ ]:
sns.scatterplot(data=notop_rules, x='support', y='confidence',hue='zhangs_metric', palette='viridis')

plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Rules with 3 Metrics Excluding Top 3 Aisles')

plt.tight_layout()
plt.show()

- Create another function to implement the steps to scale, choose the number of clusters for k means, name the scaled data, and then visualize an elbow plot and a 3D plot

In [ ]:
def cluster_and_plot(df_name, num_clusters):
    # scale the data
    df_scale = df_name[['confidence', 'support', 'zhangs_metric']].copy()
    scale_columns = ['confidence', 'support', 'zhangs_metric']
    
    scaler = StandardScaler()
    df_scale[scale_columns] = scaler.fit_transform(df_scale[scale_columns])
    
    # check elbow plot for the number of k
    inertia_values = []
    for k in range(1, 16):  # try different K values from 1 to 15
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(df_scale)
        inertia_values.append(kmeans.inertia_)
    
    plt.plot(range(1, 16), inertia_values, marker='o')
    plt.xlabel('number of clusters (K)')
    plt.ylabel('inertia')
    plt.title('elbow plot for k-means clustering')
    plt.xticks(range(1, 16))
    plt.grid(True)
    plt.show()
    
    # kmeans cluster fit
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(df_scale)
    
    # cluster numbers
    y_kmeans = kmeans.predict(df_scale)
    
    # add cluster numbers into the dataset
    df_scale['cluster'] = y_kmeans
    
    # plot scatter plot for support vs confidence
    scatter1 = plt.scatter(data=df_scale, x='support', y='confidence', c='cluster', s=50, cmap='viridis')
    center1 = kmeans.cluster_centers_
    plt.scatter(center1[:, 1], center1[:, 0], c='red', edgecolors='black', s=200, alpha=0.75)
    plt.xlabel('support level')
    plt.ylabel('confidence level')
    plt.title('clustering into {} groups: support vs confidence'.format(num_clusters))
    legend1 = plt.legend(*scatter1.legend_elements(), title='cluster')
    plt.show()
    
    # plot scatter plot for confidence vs zhang's metric
    scatter2 = plt.scatter(data=df_scale, x='confidence', y='zhangs_metric', c='cluster', s=50, cmap='viridis')
    plt.scatter(center1[:, 0], center1[:, 2], c='red', edgecolors='black', s=200, alpha=0.75)
    plt.xlabel('confidence level')
    plt.ylabel("zhang's metric level")
    plt.title('clustering into {} groups: confidence vs zhang\'s metric'.format(num_clusters))
    legend2 = plt.legend(*scatter2.legend_elements(), title='cluster')
    plt.show()
    
    # plot 3D scatter plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter3 = ax.scatter(df_scale['support'], df_scale['confidence'], df_scale['zhangs_metric'], cmap='Accent', c=y_kmeans, s=50)
    center2 = kmeans.cluster_centers_
    centers_scatter = ax.scatter(center2[:, 1], center2[:, 0], center2[:, 2], c='red', edgecolors='black', s=200, alpha=0.75, label='cluster centers')
    ax.set_xlabel('support level')
    ax.set_ylabel('confidence level')
    ax.set_zlabel("zhang's metric level")
    ax.set_title('3D scatter plot: clustering into {} groups'.format(num_clusters))
    legend3 = ax.legend(*scatter3.legend_elements(), title='cluster')
    ax.add_artist(legend3)
    ax.view_init(elev=15, azim=40)
    plt.show()

    return df_scale


In [ ]:
notop_rules_scale = cluster_and_plot(notop_rules, 7)

In [ ]:
notop_rules_scale

---

### Pruning: 2nd Round

- Again, after reviewing the pruned data and visuals, certain products are still consistently appearing across all clusters. I need to see other combinations, therefore, a second pruning

In [ ]:
# choose the name of aisles to be removed
top_aisle_list2 = ['fresh fruits','fresh vegetables','packaged vegetables fruits','packaged produce','yogurt','milk','soy lactosefreee','water seltzer sparkling water']

# top_aisle_list
top_product_list2 = df[df['aisle'].isin(top_aisle_list2)]['product_name'].tolist()

#top_product_list2

In [ ]:
# use the function we created before
remove_products_rules(rules, top_product_list2, 0.5, 'notop_rules2')

In [ ]:
sns.scatterplot(data=notop_rules2, x='support', y='confidence',hue='zhangs_metric', palette='viridis')

plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Rules with 3 Metrics Excluding 8 Aisles')

plt.tight_layout()
plt.show()

In [ ]:
notop_rules_scale2= cluster_and_plot(notop_rules2, 7)

---

---

# 4. Product Baskets

### High Frequency Basket 1
These baskets below are high in support, mostly below average in condidence, and across all zhang's metric  
Basket 1 is the most popular combinations for Wholefoods, the king of all kings

In [ ]:
# create a function to show a word cloud based on the cluster number and df
def generate_combined_wordcloud(df, rules, cluster_numbers, custom_stopwords, title=""):
    # pick cluster data for the specified cluster numbers
    combined_data = pd.DataFrame(columns=['antecedents', 'consequents'])
    for cluster_number in cluster_numbers:
        cluster_indices = df[df['cluster'] == cluster_number].index
        rules_cluster = rules.loc[cluster_indices]
        combined_data = pd.concat([combined_data, rules_cluster[['antecedents', 'consequents']]])

    # convert each frozenset of 2 cols to set, then combine them as one set in a new col called combined_set
    combined_data['combined_set'] = combined_data.apply(lambda row: set(row['antecedents']) | set(row['consequents']), axis=1)

    # drop duplicate sets
    combined_data.drop_duplicates(subset=['combined_set'], inplace=True)

    # convert combined_set from set to string
    combined_data['combined_set_str'] = combined_data['combined_set'].apply(lambda x: ', '.join(x))

    # generate text data for word cloud
    text_data = ' '.join(combined_data['combined_set_str'])

    # create stopwords
    stopwords = set(STOPWORDS)
    stopwords.update(custom_stopwords)

    # generate word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis',
                          stopwords=stopwords, collocations=False, max_words=80)
    wordcloud.generate(text_data)

    # plot word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud)
    plt.title(f'{title} - Cluster Numbers: {", ".join(map(str, cluster_numbers))}')
    plt.axis('off')
    plt.show()
    
# eg: generate_combined_wordcloud(rules_scale, rules, [3], custom_stopwords, title="High Frequency Basket 1 for 'rules'")

- All the stop words in the lists below are to prevent the products showing again from previous baskets

In [ ]:
  # some other frequent stopwords 
custom_stopwords = ['Organic', 'Bag', 'Hass', 'Baby', 'Half','Yellow','Total']

In [ ]:
generate_combined_wordcloud(rules_scale, rules, [3], custom_stopwords, title="High Frequency Basket 1 for 'rules'")

- Cluster 3 in dataframe 'rules' (before the pruning) are the best groups for high frequency products：
> Organic produce such as banana, strawberry, raspberry, spinach, avocado and lemon are the best sellers
- As long as we keep the price constantly reasonable, consumers will continue buying those products
- Although not shown in the word cloud, most products are organic, and this is consistent across all the plots below

In [ ]:
# create a function to view the rules based on number of cluster
def rules_by_clusters(cluster_numbers, rules_scale, rules):
    combined_cluster_data = pd.DataFrame(columns=['antecedents', 'consequents'])
    for cluster_number in cluster_numbers:
        cluster_indices = rules_scale[rules_scale['cluster'] == cluster_number].index
        rules_cluster = rules.loc[cluster_indices]
        combined_cluster_data = pd.concat([combined_cluster_data, rules_cluster[['antecedents', 'consequents']]])

    # convert each frozenset of 2 columns to set, then combine them as one set in a new column called combined_set
    combined_cluster_data['combined_set'] = combined_cluster_data.apply(lambda row: set(row['antecedents']) | set(row['consequents']), axis=1)

    # drop duplicate sets
    combined_cluster_data.drop_duplicates(subset=['combined_set'], inplace=True)

    return combined_cluster_data[['antecedents','consequents']]

# eg: rules_by_clusters([3],rules_scale, rules)

In [ ]:
rules_by_clusters([3],rules_scale, rules)

### High Frequency Basket 2 
This basket excludes products from top 3 aisles (no top sellers)

In [ ]:
generate_combined_wordcloud(notop_rules_scale,notop_rules, [3],custom_stopwords, title="High Frequency Basket 3 Excluding Top 3 Aisles for 'notop_rules'")

In [ ]:
rules_by_clusters([3],notop_rules_scale, notop_rules)

### High Frequency Basket 3 

In [ ]:
  # some other frequent stopwords 
custom_stopwords2 = ['Organic', 'Bag', 'Hass', 'Baby', 'Half','Yellow','Total','stage','food','bar','bars','banana','fruit','apple','whole','string']

In [ ]:
generate_combined_wordcloud(notop_rules_scale2,notop_rules2, [4], custom_stopwords2,title="High Frequency Basket 3 Excluding 8 Aisles for 'notop_rules2'")

In [ ]:
rules_by_clusters([4],notop_rules_scale2, notop_rules2)

In [ ]:
# just checking...
rules.loc[2378]

---

### High Association Basket 1
High zhang's metric, and above average confidence, but low in support  
Specific customer groups will like this combination

In [ ]:
custom_stopwords3 = ['Organic', 'Bag', 'Hass', 'Baby', 'Half','Yellow','Total','stage','food','bar','bars',
                     'banana','fruit','apple','whole','free','fruit','extra','fat','oh','non','icelandic','strained','greek'] #'greek','strained','yogurt','water','sparkling'

In [ ]:
generate_combined_wordcloud(notop_rules_scale,notop_rules, [0], custom_stopwords3, title="High Association Basket Excluding Top 3 Aisles for 'notop_rules'")

- If we can offer discounts on some of the products listed below, we can increase sales of other related products
- Different flavors or varieties of yogurt are often purchased together
- The same applies to different types of pepper, different flavors of sparkling water, snacks, yogurt smoothie, etc

In [ ]:
rules_by_clusters([0],notop_rules_scale, notop_rules)

In [ ]:
# just checking...
rules.loc[58587	]

### High Association Basket 2

In [ ]:
custom_stopwords4 = ['Organic', 'Bag', 'Hass', 'Baby', 'Half','Yellow','Total','stage','food','bar','bars',
                     'banana','fruit','apple','whole','free','fruit','extra','fat','greek','yogurt'] #,'strained',,'water','sparkling'

In [ ]:
generate_combined_wordcloud(notop_rules_scale2,notop_rules2, [3], custom_stopwords4, title="High Association Basket Excluding Top 3 Aisles for 'notop_rules2'")

In [ ]:
rules_by_clusters([3],notop_rules_scale2, notop_rules2)

In [ ]:
# just checking...
rules.loc[31351]

---

### Specific Pattern Basket 1
High in zhang's metric, but low in support and confidence  

In [ ]:
custom_stopwords5 = ['Organic', 'Bag', 'Hass', 'Baby', 'Half','Yellow','Total','stage','food','bar','bars',
                     'banana','fruit','apple','whole','free','fruit','extra','fat',
                    'greek','strained','yogurt','water','sparkling','strawberry','blueberry'] 

In [ ]:
# high assiciation but low confidence
generate_combined_wordcloud(notop_rules_scale,notop_rules, [2], custom_stopwords5, title="Specific Pattern Basket for 'notop_rules'")

- In this basket - apart from fruits and vegetables - although not purchased frequently, have a certain group of people who tend to buy these items: 
> cilantro, milk, cheese, hummus, yogurt, cream, bread, etc  
> popular flavors or types for this group are blueberry, strawberry, lowfat, etc

In [ ]:
rules_by_clusters([2],notop_rules_scale, notop_rules)

In [ ]:
# just checking...
rules.loc[61242]

### Specific Pattern Basket 2

In [ ]:
# remove cheese etc, because the previous plot already has it
custom_stopwords6 = ['Organic', 'Bag', 'Hass', 'Baby', 'Half','Yellow','Total','stage','food','bar','bars','banana','fruit','apple',
                     'whole','free','fruit','extra','fat','food'] #,'cheese','butter','string','milk','cream','mozzarella','shredded','cheddar'

In [ ]:
# high assiciation but low confidence
generate_combined_wordcloud(notop_rules_scale2,notop_rules2, [0], custom_stopwords6, title="Specific Pattern Basket for 'notop_rules2'")

In [ ]:
rules_by_clusters([0],notop_rules_scale2, notop_rules2)

In [ ]:
# just checking...
rules.loc[56716]

---

### High Frequency + Low Frequency Baskets
The "baskets" here refer to using popular products to promote less popular products

In [ ]:
# choose products with higher than 10 count to remove the extremely low count products
product_counts_df_10 = product_counts_df[product_counts_df['count'] >= 10]
#product_counts_df_10

# choose popular products (in top 95 percentile) 
percentile_top5p = product_counts_df_10['count'].quantile(0.95) # higher than 140 count
product_counts_df_top5p = product_counts_df_10[product_counts_df_10['count'] >= percentile_top5p]
#product_counts_df_top5p

# choose unpopular products (in bottom 60 percentile) 
# (the distribution of the count of products is a super long tail, most of the products are pruchased less than 100 counts)
percentile_bot60p = product_counts_df_10['count'].quantile(0.60) # lower than 26 count
product_counts_df_bot60p = product_counts_df_10[product_counts_df_10['count'] <= percentile_bot60p]
#product_counts_df_bot60p

# convert all the top 5% and bottom 60% product names to lists
product_top5p_list = product_counts_df_top5p['product_name'].tolist()
product_bot60p_list = product_counts_df_bot60p['product_name'].tolist()

In [ ]:
# create a function to choose the rules with either the antecedents or consequents include a top product, and either of them also has a bottom product
# but not with both antecedents and consequents having top products or bottom products

def top_bot_products_rules(rules, top_product_list, bot_product_list, zhangs_threshold):
    def contains_top_product(fset):
        return any(item in fset for item in top_product_list)

    def contains_bot_product(fset):
        return any(item in fset for item in bot_product_list)
    
    # need to firstly choose rules that contain bot products, use apply() to pass the values in 'antecedents' and 'consequents' to the function
    antecedents_bot_contains = rules['antecedents'].apply(contains_bot_product)
    consequents_bot_contains = rules['consequents'].apply(contains_bot_product)

    # choose the rows with either antecedents_bot_contains or consequents_bot_contains, but not both at the same time   
    bot_rules_all_zhangs = rules[(antecedents_bot_contains & ~consequents_bot_contains) | (~antecedents_bot_contains & consequents_bot_contains)]
    
    
    # secondly, choose rules that contain top products from the previous df to make sure that either antecedents or consequents have a bottom product and a top product
    antecedents_top_contains = bot_rules_all_zhangs['antecedents'].apply(contains_top_product)
    consequents_top_contains = bot_rules_all_zhangs['consequents'].apply(contains_top_product)
    
    # choose the rows with those values
    top_rules_all_zhangs = bot_rules_all_zhangs[(antecedents_top_contains | consequents_top_contains)]

    # choose only rules with zhangs metric higher than the threshold
    bottop_rules = top_rules_all_zhangs[top_rules_all_zhangs['zhangs_metric'] > zhangs_threshold]

    # assign the filtered DataFrame to the specified name
    #globals()[df_name] = bottop_rules

    return bottop_rules

# eg: bottop_rules2 = top_bot_products_rules(rules, product_top5p_list, product_bot60p_list, 0.6)

In [ ]:
bottop_rules1 = top_bot_products_rules(rules, product_top5p_list, product_bot60p_list, 0.8)

In [ ]:
# with a zhang's metric of 0.8, there are a hundred rules
bottop_rules1.head(40)

- All combinations are high in association, each combination consists of one popular product and one unpopular product. 
- Therefore, placing the unpopular products next to the popular ones should lead to faster sales for the unpopular items
- Although some combinations are duplicated, we can identify which ones should be put on sale (choose higher metrics, then the antecedent)

---

---

# 5. Recommendation

---

---

# 6. Improvement